In [94]:
import os
import json

from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet as swn

import pandas as pd
import pickle
import numpy as np

In [5]:
os.chdir('./../airbnb-data')

In [14]:
with open('voca_freq_dic.json') as fp:
    voca_freq_dic = json.load(fp)

In [19]:
voca_freq_dic2 = {}
for k,v in voca_freq_dic.items():
    if v>9:
        voca_freq_dic2[k] = v
    else: 
        continue

In [20]:
len(voca_freq_dic2)

22384

sentiwordnet 이용

In [21]:
voca_lst = list(voca_freq_dic2.keys())

In [28]:
voca_lst[0]

'go_V'

In [32]:
voca_lst2 = [tuple(voca.split('_')) for voca in voca_lst]

In [46]:
voca_lst2[:10]

[('go', 'V'),
 ('dormitory', 'N'),
 ('friend', 'N'),
 ('liked', 'J'),
 ('room', 'N'),
 ('nice', 'N'),
 ('cleanthis', 'N'),
 ('first', 'J'),
 ('time', 'N'),
 ('use', 'V')]

In [59]:
word_sent_dic = {}
word_not_in_wordnet = []
for voca in voca_lst:
    v,tag = voca.split('_')
    try:
        if tag == 'J': # adj인 경우
            senti_syn = swn.senti_synset(v+'.'+'a'+'.01')
        else:
            senti_syn = swn.senti_synset(v+'.'+tag.lower()+'.01')
            
        word_sent_dic[voca] = ( senti_syn.pos_score(),-senti_syn.neg_score() )
    except Exception as e:
        
        word_not_in_wordnet.append(voca)
        word_sent_dic[voca] = ( 0,0 )

In [60]:
len(word_not_in_wordnet)

13909

In [71]:
word_sent_dic['recommend_V']

(0.0, -0.125)

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet,ElasticNetCV
from sklearn.metrics import r2_score

In [73]:
%whos

Variable              Type                        Data/Info
-----------------------------------------------------------
fp                    TextIOWrapper               <_io.TextIOWrapper name='<...>ode='r' encoding='cp949'>
json                  module                      <module 'json' from 'C:\\<...>\lib\\json\\__init__.py'>
k                     str                         thanksnico_N
os                    module                      <module 'os' from 'C:\\Us<...>\\Anaconda3\\lib\\os.py'>
pd                    module                      <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
senti_syn             SentiSynset                 <mangosteen.n.01: PosScore=0.0 NegScore=0.0>
swn                   SentiWordNetCorpusReader    <SentiWordNetCorpusReader<...>\\corpora\\sentiwordnet'>
tag                   str                         N
tmp                   generator                   <generator object WordNet<...>ts at 0x000001DC5FF6EFC0>
v                     str         

In [75]:
with open('W_7_100.mat','rb') as fp:
    W_7_100 = pickle.load(fp)

In [88]:
len(W_7_100['w_0'])

5543

In [79]:
with open('pos_review.json') as fp:
    review = json.load(fp)

In [80]:
key_lst = list(review.keys())
sampled_key_lst = key_lst[:100]
review2 = {}
for k in sampled_key_lst:
    review2[k] = review[k]

In [83]:
ratings_100 = []
for k,rev_lst in review2.items():
    for rev in rev_lst:
        ratings_100.append(rev[0])

In [85]:
len(ratings_100)

5543

In [97]:
ratings_100 = np.asarray(ratings_100)

In [98]:
x_train, x_test, y_train, y_test= train_test_split(W_7_100['w_0'], ratings_100, test_size=0.3, random_state =0)

In [99]:
x_train.shape, y_train.shape

((3880, 22384), (3880,))

In [101]:
regr = ElasticNet(random_state=0)

In [102]:
regr.fit(x_train, y_train)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=181224, selection='cyclic', tol=0.0001,
      warm_start=False)

In [103]:
print(regr.coef_.sum()) #??

0.0


In [104]:
cv_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, .995, 1], eps=0.001, n_alphas=100, fit_intercept=True, 
                        normalize=True, precompute='auto', max_iter=2000, tol=0.0001, cv=5, 
                        copy_X=True, verbose=0, n_jobs=-1, positive=False, random_state=None, selection='cyclic')

In [106]:
cv_model.fit(x_train, y_train)

In [ ]:
print('Optimal alpha: %.8f'%cv_model.alpha_)
print('Optimal l1_ratio: %.3f'%cv_model.l1_ratio_)
print('Number of iterations %d'%cv_model.n_iter_)

In [ ]:
model = ElasticNet(l1_ratio=cv_model.l1_ratio_, alpha = cv_model.alpha_, max_iter=cv_model.n_iter_, fit_intercept=True, normalize = True)
model.fit(x_train, y_train)

In [ ]:
print(model.coef_)

In [ ]:
print(r2_score(y_train, model.predict(x_train)))

In [ ]:
print(model.predict(x_test))

In [ ]:
my_dict = {"test":y_test,"predict":model.predict(x_test)}

In [ ]:
df1 = pd.DataFrame(y_test)

In [ ]:
predict = model.predict(x_test)

In [ ]:
print(model.predict(x_test))

In [ ]:
#회귀계수
predict = model.predict(x_test)

In [ ]:
predict = predict.tolist()

In [ ]:
len(predict)